In [ ]:
import os.path as path
import raw_data_preprocessing as rdp
import warnings

warnings.filterwarnings('ignore')

googledrive_loc = '/datasets/googledrive'
data_folder_loc = 'Yi_UCI_research/GSR other works/2020 Summer_predict individual training/data/'
file_name = 'raw data/yi feng larger dataset.csv'
data_path = path.join(googledrive_loc, data_folder_loc, file_name)
raw_data = rdp.read_raw_data(data_path)
print('Used Feature columns: {}'.format(rdp.feature_col_names))
data_no_missing = rdp.delete_missing_row(raw_data)


Used Feature columns: ['Sleep', 'SES', 'VGQHoursSumPastYear', 'CFQ', 'TCA', 'Workmastery', 'Grit', 'Ambition', 'PSS', 'Competitiveness', 'MRpre', 'IC_pre', 'WM_pre', 'Updating_acc_zscore', 'Person_extraver', 'Person_agreeable', 'Person_conscien', 'Person_emot', 'Person_opennes']
The sample size of raw data is 281.
The sample size of no missing data is 262.


In [ ]:
import regressor as reg
import cluster as clu
import intervention_data_preprocessing as idp
int_test = idp.InterventionProcessor(data_no_missing)
int_test.basic_analyze()
piece_lin_reg = reg.PiecewiselinRegressor(['knot', 'slope1', 'slope2'])
int_test.register_regressor(piece_lin_reg)
int_test.fit()
# int_test.mark_outlier(['max', 'std'])
kmeans_3com = clu.Kmeans(3, ['knot', 'slope1', 'slope2'])
em_3com = clu.EM(3, ['knot', 'slope1', 'slope2'])
int_test.register_cluster_model(em_3com)
int_test.cluster()

The sample size before delete crazy sub: 262
The sample size after delete crazy sub: 243
The mean r2 for <class 'regressor.PiecewiselinRegressor'> is: 0.738
The silhouette score of GaussianMixture(n_components=3, random_state=0) is: 0.244
Sample of each cluster: label
0     86
1    104
2     53
dtype: int64
The mean of each mixture component: [[-1.08199669  0.74662378  0.12813143]
 [ 0.15102369 -0.23861044  0.02514202]
 [ 1.40739074 -0.69178856 -0.2561036 ]]


In [ ]:
int_test.data

,ID,WmeanN_1,WmeanN_2,WmeanN_3,WmeanN_4,WmeanN_5,WmeanN_6,WmeanN_7,WmeanN_8,WmeanN_9,WmeanN_10,max,std,knot,slope1,slope2,label,cluster_0,cluster_1,cluster_2
0,RGTA104,2.532189,2.951220,3.981067,3.069697,6.489933,8.599343,3.490166,5.315956,6.595960,6.297114,8.599343,2.021831,6.0,0.828884,-0.132875,1,3.931761e-23,1.000000,2.735127e-11
1,RGTA107,2.004138,2.490342,2.164074,2.579515,4.816541,4.362275,4.104225,2.912553,4.726727,4.812147,4.816541,1.169164,5.0,0.544715,0.112200,1,2.181939e-14,1.000000,1.456749e-15
2,RGTA110,3.025449,3.758287,3.861799,3.580940,4.017241,5.250646,4.645780,5.345315,4.882199,4.956466,5.345315,0.787565,8.0,0.294313,-0.164472,1,3.245068e-54,0.541341,4.586587e-01
3,RGTA113,2.552137,2.758261,3.335548,3.615517,3.603839,4.394864,4.600334,4.544872,5.965574,5.740260,5.965574,1.153073,8.0,0.335222,0.560211,2,5.745222e-76,0.005349,9.946510e-01
4,RRTA101,1.738197,2.528198,2.299090,2.566576,3.322917,3.460189,3.578313,3.400264,3.597671,2.701987,3.597671,0.639926,7.0,0.309710,-0.261232,1,2.326919e-37,0.998456,1.543968e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,RLB103,2.468144,3.242165,2.601648,2.430894,2.463190,2.468927,2.739938,2.109661,3.051988,3.321937,3.321937,0.394269,8.0,-0.057341,0.514140,2,2.511814e-74,0.021809,9.781906e-01
264,RLB104,5.124611,8.013245,4.577640,4.939683,4.714734,5.131902,5.109467,4.384393,4.742297,5.158046,8.013245,1.027339,5.0,-0.353524,0.022224,1,4.590835e-12,1.000000,4.060199e-12
265,RLB115,3.727273,3.825243,4.129630,3.869416,4.261146,4.506289,5.222930,5.200658,5.336508,NaN,5.336508,0.645941,4.0,0.082159,0.293587,1,6.819024e-10,1.000000,1.347509e-20
266,RLB120,4.619898,4.160526,5.081921,4.915344,5.580475,5.324324,3.191601,5.307087,4.853846,5.693878,5.693878,0.746181,7.0,-0.038382,0.321412,1,3.203855e-46,0.982555,1.744464e-02


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b8ead81-70bf-4d6b-bd50-8194f8f72b46' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>